In [538]:
import os
import json
from IPython.display import HTML, display
try:
    (notebook_dir, e)
except:
    e = "a --- b"
    notebook_dir = "."

In [539]:
padding = {
    "space": 275,  # Standard space width
    "ks_": -180,
    "ts_": -380,
    "ka_": -280,
    "ko_": -390,
    "ta_": -250,
    "to_": -250,
    "tb_": -250,
    "tc_": -280,
    "tuR_": -340,
}

### Nommage des fichiers : 
* "ta_" suffit si la lettre attendue est \_a; a_ attend un a à gauche (ou une lettre compatible), pareil dans l'autre sens.
* uL et uR (Left; Right par rapp à la direction vers laquelle l'arche pointe); bL et bR (direction de la pointe pour les heads) ; les autres lesttres sont en a_left; a_right; etc
* Les autres glyphes sont notés comme leur composante principale (par ex "ks") puis un commentaire : ks_above_t

### TODO :
* Découper uL et uR en deux formes, avec des connecteurs "avant" et "après" pour interfacer avec fa_ et n. Revoir le design de la lettre... Créer les jonctions (t, s) en un glyph bien défini.
* Voyelles précédent les clusters (et f)
* Formes T basses (mécanique "d", "g" et "s+...")
* Diacritiques (i, l, h)
* Q

In [540]:
fea = '''

table GDEF {
    GlyphClassDef
        [], # Base glyphs
        [], # Ligature glyphs
        [j h _i_ _j _h ji_ hi_ la_ lo_ _al _ol _sla_ _slp_ _bRl _bLl lbR_ lbL_], # Mark glyphs
        []; # Component glyphs
} GDEF;

feature ccmp {
    script DFLT;
    language dflt required;
    script latn;
    language dflt required;

    ######################################################################### 1 CHOIX CONTEXTUEL
    
    lookup ETAPE_1_REPLACEMENTS {

        sub m by n;
        
    } ETAPE_1_REPLACEMENTS;
    
    ################################
    
    lookup ETAPE_1_PRE_MERGE {
        
        sub j i by ji;
        sub j j by ji;
        sub h i by hi;
        sub h j by hi;
        
        # Lettre S
        
        sub s i by si_;
        
    } ETAPE_1_PRE_MERGE;
    
    #################################
    
    lookup ETAPE_1_FLOTTANTES {  # Décide si les lettres flottantes doivent devenir des diacritiques ou non

        # IJ positioning
        
        sub s i' by s _i_;

        sub [a o e p y bL bR uL uR n] i' [a o e p y bL bR uL uR n] by _i_;  # i becomes a diacritic
        sub i' [n] by _i_;
        
        sub ji' [n a o e p y bL bR uL uR n] by ji_;  # ji becomes a diacritic if followed by something
        sub hi' [n a o e p y bL bR uL uR n] by hi_; 
        
        sub j' [a o e p y bL bR uL uR] by j;
        sub j' by _j;  # If not followed
        
        # H positioning
        
        sub h' [a o e p y bL bR uL uR] by h;  # Check if followed by something
        sub [a o e p y bL bR uL uR] h' by _h;

        # L positioning
        
        sub [s t d] l' [a e bR uR] by _sla_;  # s t k precedence
        sub [s t d] l' [bL uL p] by _slp_;
        
        sub l' [a e] by la_;  # If followed by something
        sub l' [o y] by lo_;
        sub l' [bL uL p] by lbL_;
        sub l' [bR uR] by lbR_;
        
        sub [a e] l' by _al;  # If not followed
        sub [o y] l' by _ol;
        sub [bL uL p] l' by _bLl;
        sub [bR uR] l' by _bRl;
        
    } ETAPE_1_FLOTTANTES;
    
    #################################

    lookup ETAPE_1_NON_FLOTTANTES {  # Choix forme contextuelle de base
        
        lookupflag IgnoreMarks;

        # Lettre F

        sub [a p] f' by fa_;
        sub [o e] f' by fo_;

        sub f' [a c u] by fa_;
        sub k' [a c u b] by ka_;

        sub f' [o b] by fo_;  # This isnt optimal but it's easier to read
        sub k' [o b] by ko_;

        # Lettre B

        sub [o n fo_] b' by bR;
        sub b' by bL;

        # Lettre U

        sub [t s] u' by uR;  # Ignore next letter
        sub [n] u' by uL;  
        sub u' [a e p o b c y n] by uR;
        sub u' [f] [a e p c u n] by uR;
        sub u' by uL;

    } ETAPE_1_NON_FLOTTANTES;

    ######################################################################### 2 DECOMPOSITION FUSION
    
    lookup ETAPE_2_DECOMPOSITION {  # Decoupe tous les caracteres en composants

        lookupflag IgnoreMarks;

        # Explicit 2 Parts

        sub a by a_ _a;
        sub o by o_ _o;
        sub e by a_ _e;
        sub p by a_ _p;
        sub y by o_ _y;
        sub c by c_ _c;
        sub s by s_ _s;
        
        # Implicit 2 Parts
        
        sub si_ by si_ _s;
        
        # B
        
        sub bR by o_ _b;
        sub bL by b_ _o;

    } ETAPE_2_DECOMPOSITION;

    #################################

    lookup ETAPE_2_FUSION {  # Fusionne les composants des que possible

        lookupflag IgnoreMarks;

        # AO OA

        sub _a o_ by _ao_;
        sub _o a_ by _oa_;

        sub _oa_ a_ by _oa_;
        sub _ao_ o_ by _ao_;

        # S

        sub _s a_ by _sa_;
        sub _s b_ by _sb_;
        sub _s c_ by _sc_;
        
        sub _s uR by uR;

        # K

        sub k s_ by ks_;

        # T

        sub t s_ by ts_;
        sub t a_ by ta_;
        sub t o_ by to_;
        sub t b_ by tb_;
        sub t c_ by tc_;

    } ETAPE_2_FUSION;

    ######################################################################### 3 ESTHETIQUE

    lookup ETAPE_3_ESTHETIQUE {  # Applique les formes purement esthétiques

        lookupflag IgnoreMarks;
        
        # HORIZONTAL CONNECTORS

        sub [n fo_] a_' by _na_;
        sub [n fo_] c_' by _nc_;
        sub [n fa_] o_' by _no_;
        sub [_a _p] b_' by _bn_;
        
        sub _c' [n fo_] by _cn_;
        sub _a' [n fo_] by _an_;
        sub _o' [n fa_] by _on_;
        
        sub _p' [n fo_] by _pn_;
        sub _e' [n fo_ s_ si_] by _en_;
        sub _b' [n fa_] by _bn_;

        # U FORMS

        sub ka_ uR' by uR_after_high;
        sub _s uR' by uR_after_high;

        sub n uL' by uL_after_n;
        sub [ka_ fa_] uL' by uL_after_high;
        
        sub uR' n by uR_before_n;
        sub uR' [fa_] by uR_before_high;

        sub t' uR by tuR_;

        # B ENDING FORMS

        sub [fo_] b_' _o' [a_ o_ s_ si_] by b_; # Ignoring certain combinations
        sub _e' b_' _o' by b_ending;
        sub _o o_' _b' by b_ending;  # _b implies it is not connected to the next letter
        # TODO : Mix b_ending to bs, bn, bf, bt, bk depending on the consonnant that follows
        # TODO : Ignore b_ending if followed by any other shit

        # S ENDING FORMS

        sub [_a n fa_] s_' _s' by s_after_high;
        sub [fo_ _en_] s_' _s' by s_after_low;
        
        sub _o' s_' _s' by os_ending;
        sub _c' s_' _s' by cs_ending;
        sub _p' s_' _s' by ps_ending;
        sub _y' s_' _s' by ys_ending;

    } ETAPE_3_ESTHETIQUE;

    ######################################################################### 4 KERNING

    lookup ETAPE_4_KERNING {

        lookupflag IgnoreMarks;

        # MERGE VOYELLE -80

        pos [_a _o] [a_ o_] -80;
        pos _p [a_ o_] -90;
        pos _o b_ending -5;

        # VOYELLE U

        pos tuR_ [uR uR_before_n uR_before_high] -160;
        pos [s_ si_] [uR uR_before_n uR_before_high] -120;

        # MERGE VOYELLE S -80

        pos [_s] [o_] -80;
        pos _a s_after_high -80;
        pos [fo_ _en_] s_after_low -80;

        pos [n fa_] s_after_high -5;

        # MERGE VOYELLE F N

        pos [_a _o _p _an_ _on_ _cn_ _pn_ _bn_ _en_] [fa_ fo_ n _bn_] -20;
        pos [fa_ fo_ n] [a_ o_ _na_ _no_ _nc_] -20;

        # POSITIONNEMENT K T VOYELLE

        pos ka_ c_ -40;

        # MERGE K T S
        
        

    } ETAPE_4_KERNING;
    
    #################################

    lookup ETAPE_4_DIACRITIC_KERNING {
        
        # POSITIONNEMENT IJHL NON DIACRITIQUES

        pos i [ts_ ta_] <-80 480 0 0>;
        pos i [s_ si_] <-80 420 0 0>;
        pos [i ji] [a_ o_] <-80 0 0 0>;
        
        # VOYELLE I
        
        pos _i_ [n] <-260 360 0 0>;  # x-advance y-advance x-placement y-placement (placement moves the glyph)
        pos _i_ [a_ o_ _na_ _no_ _ao_ _oa_ _a _o] <-290 -260 0 0>;

        # FLOTTANTES JH

        # pos [s_ si_] [j h] <180 0 0 0>;
        pos [j h ji_ hi_] [n] <-100 360 0 0>;
        pos [j h ji_ hi_] [a_ o_ _na_ _no_] <180 360 0 0>;
        pos [j h ji_ hi_] [_a _o _oa_ _ao_] <-100 360 0 0>;
        
    } ETAPE_4_DIACRITIC_KERNING;
    
    #################################
    
    lookup ETAPE_4_CONSTANT_KERNING {

        # POSITIONNEMENT S
        
        pos s_ <0 0 -80 0>;
        pos si_ <0 0 -140 0>;

        # POSITIONNEMENT IJH
        
        pos [_j _h] <-375 -260 0 0>;
        pos [i ji hi] <60 100 40 0>;
        
        # POS L
        
        pos [la_ lo_ lbR_ lbL_] <0 420 0 0>;
        pos [_slp_ _sla_] <-260 420 0 0>;
        pos [_al _ol] <-550 -140 0 0>;

    } ETAPE_4_CONSTANT_KERNING;

} ccmp;
'''

In [541]:
try: counter += 1
except: counter = 0
if not os.getcwd().endswith('bin'): 
    notebook_dir = os.getcwd()  # Actual working directory
    %cd ".\spetekkimyo\ffpython\bin\"
json.dump(padding, open(f'{notebook_dir}\\spetekkimyo\\input\\padding.json', 'w'))
with open(f'{notebook_dir}\\spetekkimyo\\input\\features.fea', 'w') as f: f.write(fea)
!"ffpython.exe" "..\..\generate.py" "{notebook_dir}\output\test.otf"

Imported 103 glyphs: a a_ b bL bR b_ b_ending c cs_ending c_ d d_ e f fa_ fo_ h hi hi_ i j ji ji_ k ka_ ko_ ks ks_ l la_ lbL_ lbR_ lo_ m n o os_ending o_ p ps_ending s si_ space s_ s_after_high s_after_low t ta_ tb_ tc_ tks_ tk_high tk_low to_ ts_ tuR_ u uL uL_after_high uL_after_n uR uR_after_high uR_before_high uR_before_n y ys_ending _a _al _an_ _ao_ _b _bLl _bn_ _bRl _c _cn_ _da_ _e _en_ _fc _h _i_ _j _na_ _nc_ _no_ _o _oa_ _ol _on_ _p _pn_ _s _sa_ _sb_ _sc_ _si_ _sj_ _sla_ _slp_ _ti_ _y _yn_
Imported features
Font generated at d:\Github\spetekkimyo\output\test.otf


I'm sorry this file is too complex for me to understand (or is erroneous, or is empty)


In [ ]:
e = """
<span class="ss">na ku </span> pb
""" # TODO : jubn (predefined _bn and _bf charecters) ; da - sta - sda ; fta fsa fka ; ga - ska ; spacing inter mots et "spe" ; jonction u + autres lettres

In [543]:
html = """<style>p {font-size: 120px;} @font-face {src:url('./output/test.otf?version=%s');font-family:'test';} .ss {font-family:'test';}</style>
<p>%s</p>""" % (counter, e)
display(HTML(html))

### Guidelines

Epaisseur du trait : 80

Largeur : **550** (275) -- consonants 320 - 440 (max d'épaisseur) - 550 (pointe droite)

Hauteur standard : 320 -- max = 730 -- low = -240

Départ : h=-12